In [1]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers
import time

In [21]:
configs = {
    "http": "192.168.50.107:29200",
    "span_index":"dc_prophet_test_*"
}

In [3]:
24*60*60*7

604800

In [24]:
class Detect(object):
    def __init__(self, config):
        self.config = config
        self.es = Elasticsearch(self.config['http'])
         
    def scroll_search(self, body, index):
        batches = 10000  # 每一批数据量大小
        queryData = self.es.search(index=index, scroll='5m', timeout='60s', size=batches, body=body)
        metadata = queryData.get("hits").get("hits")
        scroll_id = queryData["_scroll_id"]
        total = queryData["hits"]["total"]['value']
        for i in range(int(total / batches)):
            res = self.es.scroll(scroll_id=scroll_id, scroll='5m')  # scroll参数必须指定否则会报错
            metadata += res["hits"]["hits"]
        self.es.clear_scroll(scroll_id=scroll_id)
        return metadata
    
    def job(self):
        body = {
                    'query': {
                        'bool': {
                            'must': [{'range': {'timestamp': {'gte': int(time.time()) - 604800}}}
                                     ]
                        }
                    }
                }

        metadata = self.scroll_search(body, self.config['span_index'])
        return metadata

In [25]:
detect = Detect(configs)
metadata = detect.job()